In [ ]:
using CABLAB
using CABLABPlots

In this Notebook we want to visualize the regions where cube variable take on their highest or lowest value. To do this we calculate the 99% and the 1% quantile for each variable from a subset of the data and afterwards count how often the quantile was exceeded for each variable in each grid cell.

In [ ]:
c             = Cube("/home/jovyan/work/datacube/cablab-datacube-1.0.0/low-res/")
vars          = ["air_temperature_2m","soil_moisture","c_emissions"];
cdata         = getCubeData(c,latitude=(30,65), longitude=(-10,40),variable=vars); 
sampledata    = sampleLandPoints(cdata,2000);

Get the quantiles for the subsample

In [ ]:
quants=mapCube(timespacequantiles,sampledata,[0.01,0.99])

Define a function that counts quantile crossings for each grid cell

In [ ]:
"""
How often the upper or lower quantiles are crossed in each time series
"""
function countExtremes(xout::AbstractArray,xin::AbstractVector,qvec::AbstractVector)
    nlow,nhigh=0,0
    qlow,qhigh=qvec
    for v in xin
        if !isnan(v)
            v<=qlow && (nlow+=1)
            v>=qhigh && (nhigh+=1)
        end
    end
    xout[1]=nlow
    xout[2]=nhigh
end

registerDATFunction(countExtremes,((TimeAxis,),(QuantileAxis,)),(QuantileAxis,),inmissing=(:nan,:nan),outmissing=:nan,no_ocean=1);

In [ ]:
@time o=mapCube(countExtremes,(cdata,quants),outtype=(Int,), no_ocean=true)

Now we plot the low extremes

In [ ]:
for i=1:3
display(o.axes[4].values[i])
display(plotMAP(o,dmax=10,quant=1,var=i))
end

And the high extremes

In [ ]:
for i=1:3
display(o.axes[4].values[i])
display(plotMAP(o,dmax=10,quant=2,var=i))
end